In [1]:
import sys
import warnings

sys.path.append("..")
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from model.basic.collaborative_filtering.memory_based import UserBasedMemoryCF
from model.basic.collaborative_filtering.memory_based import ItemBasedMemoryCF
from data_builder.user_item_data import UserItemData

In [3]:
movies = pd.read_csv('../demo_data/movies.csv')
ratings = pd.read_csv('../demo_data/ratings.csv')

ratings.rename(columns={'userId': 'user_id', 'movieId': 'item_id', 'rating': 'feedback'}, inplace=True)

print("Movies DataFrame:")
print(movies.head())

print("\nRatings DataFrame:")
print(ratings.head())

Movies DataFrame:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings DataFrame:
   user_id  item_id  feedback  timestamp
0        1        1       4.0  964982703
1        1        3       4.0  964981247
2        1        6       4.0  964982224
3        1       47       5.0  964983815
4        1       50       5.0  964982931


In [4]:
user_item_data = UserItemData(ratings)
user_item_matrix = user_item_data.build_user_item_matrix()

In [5]:
recommender = UserBasedMemoryCF(user_item_matrix, similarity_metric='cosine')

user_id = 1
predicted_feedback = recommender.predict_feedback(user_id)

recommended_items = recommender.recommend_items(predicted_feedback, user_id, n_recomm=5)

movie_id_to_title = pd.Series(movies.title.values, index=movies.movieId).to_dict()

print(f"\nRecommended Movies for User {user_id}:")
for item_id, feedback_score in recommended_items:
    movie_title = movie_id_to_title.get(item_id, "Unknown Movie")
    print(f"Movie ID: {item_id}, Movie: {movie_title}, Predicted Feedback: {feedback_score:.2f}")


Recommended Movies for User 1:
Movie ID: 318, Movie: Shawshank Redemption, The (1994), Predicted Feedback: 2.65
Movie ID: 589, Movie: Terminator 2: Judgment Day (1991), Predicted Feedback: 2.09
Movie ID: 858, Movie: Godfather, The (1972), Predicted Feedback: 1.86
Movie ID: 2762, Movie: Sixth Sense, The (1999), Predicted Feedback: 1.66
Movie ID: 4993, Movie: Lord of the Rings: The Fellowship of the Ring, The (2001), Predicted Feedback: 1.62


In [6]:
user_item_data = UserItemData(ratings)
user_item_matrix = user_item_data.build_user_item_matrix()

In [7]:
item_cf_model = ItemBasedMemoryCF(user_item_matrix, similarity_metric='cosine')
item_cf_model.compute_item_similarity() 

In [8]:
user_id = 1
predicted_feedback = item_cf_model.predict_feedback(user_id)

recommendations = item_cf_model.recommend_items(predicted_feedback, user_id, n_recomm=5)

recommended_movie_ids = [item for item, _ in recommendations]
recommended_movies = movies[movies['movieId'].isin(recommended_movie_ids)]
print("Recommendations for User", user_id)
for item, feedback in recommendations:
    movie_title = recommended_movies[recommended_movies['movieId'] == item]['title'].values[0]
    print(f"Movie ID: {item}, Title: {movie_title}, Predicted Feedback: {feedback:.2f}")

Recommendations for User 1
Movie ID: 6145, Title: Venom (1982), Predicted Feedback: 1.77
Movie ID: 876, Title: Supercop 2 (Project S) (Chao ji ji hua) (1993), Predicted Feedback: 1.50
Movie ID: 61697, Title: Righteous Kill (2008), Predicted Feedback: 1.36
Movie ID: 476, Title: Inkwell, The (1994), Predicted Feedback: 1.33
Movie ID: 2812, Title: In Too Deep (1999), Predicted Feedback: 1.33
